In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import scipy.io as sio
import numpy as np
import mne
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

import torch
from torch.utils.data import Dataset, SubsetRandomSampler, DataLoader
import torch.nn as nn

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ShuffleSplit, cross_val_score
from mne.decoding import CSP
import zipfile
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['decoding-the-human-brain']


In [2]:
os.listdir('../input/decoding-the-human-brain')

['test_17_23.zip',
 'random_submission.csv',
 'train_07_12.zip',
 'train_13_16.zip',
 'train_01_06.zip']

In [ ]:
for item in os.listdir('../input/decoding-the-human-brain'):  
     if 'train' in item:
        with zipfile.ZipFile("../input/decoding-the-human-brain/{}".format(item),"r") as zip_ref:
            zip_ref.extractall("./data/{}".format(item.split('.')[0]))   #data extraction

In [ ]:
os.listdir('./data/')

In [ ]:
# l = 0
# for subjectset in os.listdir('./data/'):
#      if '13' not in subjectset:
#         for subject in os.listdir('./data/{}/data'.format(subjectset)):
#             train_mat = sio.loadmat('./data/{}/data/{}'.format(subjectset,subject))
#             X, Y = np.transpose(train_mat['X'], (0,2,1)), train_mat['y']
#             if l == 0:
#                 x_train, y_train = X[:,130:,:], Y
#                 l = 1
#             else:
#                 x_train = np.concatenate([x_train, X[:,130:,:]], axis=0)
#                 y_train = np.concatenate([y_train, Y], axis=0)

In [ ]:
# scaler = StandardScaler()
# s0, s1, s2 = x_train.shape[0], x_train.shape[1], x_train.shape[2]
# x_train = x_train.reshape(s0,s1*s2)
# x_train = scaler.fit_transform(x_train)
# dataset = np.concatenate([x_train, y_train], axis=1)
# dataset.shape

In [ ]:
# val_size = 0.1
# seed = 42
# dataset_size = len(dataset)
# indices = list(range(dataset_size))
# val_split = int(np.floor(val_size * dataset_size))
# np.random.seed(seed)
# np.random.shuffle(indices)
# train_indices = indices[val_split:]
# test_indices = indices[:val_split]

In [ ]:
# x_train, y_train = dataset[train_indices, :-1], dataset[train_indices, -1]

In [ ]:
# x_test, y_test = dataset[:val_split, :-1], dataset[:val_split, -1]

In [ ]:
# models = []
# models.append(('LR', LogisticRegression()))
# models.append(('KNN3', KNeighborsClassifier(n_neighbors=3)))
# models.append(('KNN5', KNeighborsClassifier(n_neighbors=5)))
# models.append(('NB', GaussianNB()))
# models.append(('SVC', SVC()))
# models.append(('SVM', SVC(kernel='poly')))
# models.append(('LSVC', LinearSVC()))
# models.append(('RFC', RandomForestClassifier()))
# models.append(('DTC', DecisionTreeClassifier()))

In [ ]:
# for name, model in models:
#     model.fit(x_train, y_train)
#     predictions = model.predict(x_test)  
#     acc = np.sum(y_test == predictions) / len(y_test)
#     print("Accuracy for predictions based on {} is : {:.2f}%".format(name,acc*100))

In [ ]:
class RawDataset(Dataset):
    #create dataset for models
    def __init__(self, train=True):
        t = 'train' if train else 'test'
        self.feats, self.labels = self.load_matfiles()
        self.feats = self.scale(self.feats)

    def load_matfiles(self):
        l = 0
        for subjectset in os.listdir('./data/'):
            for subject in os.listdir('./data/{}/data'.format(subjectset)):
                train_mat = sio.loadmat('./data/{}/data/{}'.format(subjectset,subject))
                #we use transpose because we want our data to be batchxseq_lenxfeatures
                X, Y = np.transpose(train_mat['X'], (0,2,1)), train_mat['y']
                if l == 0:
                    x_train, y_train = X[:,130:,:], Y  #we remove the first 0.5 seconds
                    l = 1
                else:
                    x_train = np.concatenate([x_train, X[:,130:,:]], axis=0)
                    y_train = np.concatenate([y_train, Y], axis=0)
        return x_train, y_train
    
    def scale(self, X):
        #scaling of the data 
        scaler = StandardScaler()
        s0, s1, s2 = X.shape[0], X.shape[1], X.shape[2]
        X = X.reshape(s0,s1 * s2)
        X = scaler.fit_transform(X)
        X = X.reshape(s0, s1, s2)
        return X

    def __getitem__(self, item):
        return self.feats[item], self.labels[item].astype('float32')

    def __len__(self):
        return len(self.labels)

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, train=True):
        t = 'train' if train else 'test'
        self.feats, self.labels = self.load_matfiles()
        self.feats = self.scale(self.feats)

    def load_matfiles(self):
        l = 0
        for subjectset in os.listdir('./data/'):
            for subject in os.listdir('./data/{}/data'.format(subjectset)):
                train_mat = sio.loadmat('./data/{}/data/{}'.format(subjectset,subject))
                X, Y = np.transpose(train_mat['X'], (0,2,1)), train_mat['y']
                if l == 0:
                    x_train, y_train = X[:,130:,:], Y
                    l = 1
                else:
                    x_train = np.concatenate([x_train, X[:,130:,:]], axis=0)
                    y_train = np.concatenate([y_train, Y], axis=0)
        return x_train, y_train
    
    def scale(self, X):
        scaler = StandardScaler()
        s0, s1, s2 = X.shape[0], X.shape[1], X.shape[2]
        X = X.reshape(s0 * s1, s2)
        X = scaler.fit_transform(X)
        X = X.reshape(s0, s1, s2)
        return X

    def __getitem__(self, item):
        return np.expand_dims(self.feats[item], 0), self.labels[item].astype('float32')

    def __len__(self):
        return len(self.labels)

In [ ]:
def torch_train_val_split(
        dataset, batch_train, batch_eval,
        val_size=.2, shuffle=True, seed=42):
    # Creating data indices for training and validation splits:
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    val_split = int(np.floor(val_size * dataset_size))
    if shuffle:
        np.random.seed(seed)
        np.random.shuffle(indices)
    train_indices = indices[val_split:]
    val_indices = indices[:int(val_split/2)]
    test_indices = indices[int(val_split/2):val_split]
    print("Training Set size : {}, Validation Set size : {}, Test Set size : {}".format(len(train_indices), len(val_indices), len(test_indices)))
    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)
    test_sampler = SubsetRandomSampler(test_indices)

    train_loader = DataLoader(dataset,
                              batch_size=batch_train, 
                              sampler=train_sampler)
    val_loader = DataLoader(dataset,
                            batch_size=batch_eval ,
                            sampler=val_sampler)
    test_loader = DataLoader(dataset,
                            batch_size=batch_eval ,
                            sampler=test_sampler)
    return train_loader, val_loader, test_loader

In [ ]:
raw_data = RawDataset()
train_loader, val_loader, test_loader = torch_train_val_split(raw_data, 32 ,32, val_size=.2, shuffle=True)

In [ ]:
for index,item in enumerate(train_loader):
    print(index, item[0].size(), item[1].size())
    if index == 3:
        break

In [ ]:
def train(train_loader, val_loader, model, criterion, optimizer, max_epochs=50, patience=20):
    e = 0
    min_val_loss = 1e10
    plot_train_loss, plot_val_loss = [], []
    while patience >= 0 and e < max_epochs:
        #list of train losses per epoch
        batch_loss = []
        model.train()
        for i_batch, data_batched in enumerate(train_loader, 0):
            data_batched[0] = data_batched[0].to(0)
            y_pred = model(data_batched[0]).to(0)
            #calculate batch loss 
            data_batched[1] = data_batched[1].to(0)
            loss = criterion(y_pred, data_batched[1])
            #append loss to list in order to print train loss per epoch
            batch_loss.append(loss.item())
            print('Batch loss: {}'.format(loss.item()))
            # Zero gradients, perform a backward pass, and update the weights.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        train_loss = np.mean(batch_loss)
        plot_train_loss.append(train_loss)
        print('Train loss for epoch {} : {}'.format(e,train_loss))
        # set eval mode in order to stop dropout during inference
        model.eval()
        val_loss = []
        for i_batch, data_batched in enumerate(val_loader, 0):
            data_batched[0]= data_batched[0].to(0)
            y_pred = model(data_batched[0]).to(0)
            data_batched[1] = data_batched[1].to(0)
            loss = criterion(y_pred, data_batched[1])
            val_loss.append(loss.item()) 
        validation_loss = np.mean(val_loss)
        plot_val_loss.append(validation_loss)
        print('Val loss for epoch {} : {}, Best loss : {}, Patience : {}'.format(e, validation_loss, 
                                                                                 min_val_loss, patience))
        e += 1
        if validation_loss <= min_val_loss:
            min_val_loss = validation_loss
            patience = 20
            torch.save(model.state_dict(), "LSTMmodel_params.pt")
        else:
            patience -= 1

In [ ]:
def predict(test_loader, model, criterion):
    predictions, y_test, test_loss = [], [], []
    model.load_state_dict(torch.load("LSTMmodel_params.pt"))
    # set eval mode in order to stop dropout during inference
    model.eval()
    model.to(0)
    for i_batch, data_batched in enumerate(test_loader, 0):
        data_batched[0] = data_batched[0].to(0)
        y_pred = model(data_batched[0])
        preds = y_pred.data.cpu().numpy()
        predictions += list(np.round(preds))
        # get sorted version of y_test
        data_batched[1] = data_batched[1].to(0)
        y_test += list(data_batched[1].data.cpu().numpy())
        loss = criterion(y_pred, data_batched[1]) 
        test_loss.append(loss.item()) 
    total_loss = np.mean(test_loss)
    print('Test loss: {}'.format(total_loss))
    accuracy = np.sum(np.asarray(predictions) == np.asarray(y_test)) / len(y_test)
    print("Test set accuracy is {:.2f}% ".format(accuracy*100))
    return(predictions, y_test)

In [ ]:
class DNN(nn.Module):
    def __init__(self, layer1, layer2, layer3, output_size):
        super(DNN, self).__init__()
        self.layer1 = layer1
        self.layer2 = layer2
        self.layer3 = layer3
        self.output_size = output_size
        self.fc1 = nn.Linear(self.layer1, self.layer2)
        self.fc2 = nn.Linear(self.layer2,self.layer3)
        self.fc3 = nn.Linear(self.layer3, self.output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,x):
        x = x.view(x.size(0), -1)
        x1 = self.fc1(x)
        x2 = self.fc2(x1)
        x3 = self.fc3(x2)
        output = self.sigmoid(x3)
        return output

In [ ]:
FC1, FC2, FC3, D_out = 245*306, 200, 20, 1

model = DNN(FC1, FC2, FC3, D_out)
model.to(0)

criterion =  nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), weight_decay=0.1)
model.parameters

In [ ]:
train(train_loader, val_loader, model, criterion, optimizer, max_epochs=50, patience=20)

In [ ]:
predictions, labels = predict(val_loader, model, criterion)

In [ ]:
class BasicLSTM(nn.Module):
    def __init__(self, input_dim, rnn_size, output_dim, num_layers, dropout, bidirectional=False):
        super(BasicLSTM, self).__init__()
        self.bidirectional = bidirectional
        self.feature_size = rnn_size * 2 if self.bidirectional else rnn_size
        # --------------- Insert your code here ---------------- #
        # Initialize the LSTM, Dropout, Output layers
        self.input_size = input_dim
        self.num_layers= num_layers
        self.output_size = output_dim
        self.dropout = dropout
        self.lstm = nn.LSTM(self.input_size, rnn_size, self.num_layers, dropout=self.dropout, 
                            batch_first=True, bidirectional=self.bidirectional)
        self.fc = nn.Linear(self.feature_size, self.output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        """ 
            x : 3D numpy array of dimension N x L x D
                N: batch index
                L: sequence index
                D: feature index
         """
        lstm_outs, hidden = self.lstm(x)
        last_out = self.last_timestep(lstm_outs, self.bidirectional)
        output = self.fc(last_out)
        output = self.sigmoid(output)
        return output

    def last_timestep(self, outputs, bidirectional=False):
        """
            Returns the last output of the LSTM taking into account the zero padding
        """
        if bidirectional:
            forward, backward = self.split_directions(outputs)
            last_forward = forward[:, -1, :]
            last_backward = backward[:, 0, :]
            # Concatenate and return - maybe add more functionalities like average
            return torch.cat((last_forward, last_backward), dim=-1)
        else:
            return outputs[:, -1, :]

    @staticmethod
    def split_directions(outputs):
        direction_size = int(outputs.size(-1) / 2)
        forward = outputs[:, :, :direction_size]
        backward = outputs[:, :, direction_size:]
        return forward, backward

In [ ]:
D_in, H, D_out, num_layers, dropout, biLSTM = 306, 20, 1, 2, 0.2, True

model = BasicLSTM(D_in, H, D_out, num_layers, dropout, biLSTM)
model.to(0)

criterion =  nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), weight_decay=0.001)
model.parameters

In [ ]:
train(train_loader, val_loader, model, criterion, optimizer, max_epochs=50, patience=20)

In [ ]:
predictions, labels = predict(val_loader, model, criterion)

In [ ]:
class CLSTM(nn.Module):
    def __init__(self, input_dim, rnn_size, output_dim, num_layers, dropout, bidirectional=False):
        super(CLSTM, self).__init__()
        self.bidirectional = bidirectional
        self.feature_size = rnn_size * 2 if self.bidirectional else rnn_size
        # --------------- Insert your code here ---------------- #
        # Initialize the LSTM, Dropout, Output layers
        self.input_size = input_dim
        self.num_layers= num_layers
        self.output_size = output_dim
        self.dropout = dropout
        self.conv1 = nn.Conv1d(in_channels=self.input_size ,out_channels=self.input_size ,kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(in_channels=self.input_size ,out_channels=self.input_size ,kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool1d(2)
#         self.conv3 = nn.Conv1d(in_channels=self.input_size ,out_channels=self.input_size ,kernel_size=3, padding=1)
#         self.pool3 = nn.MaxPool1d(2)
        self.lstm = nn.LSTM(self.input_size, rnn_size, self.num_layers, dropout=self.dropout, 
                            batch_first=True, bidirectional=self.bidirectional)
        self.fc = nn.Linear(self.feature_size, self.output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        """ 
            x : 3D numpy array of dimension N x L x D
                N: batch index
                L: sequence index
                D: feature index
         """
        x = x.transpose(1, 2)
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
#         x = self.conv3(x)
#         x = self.pool3(x)
        x = x.transpose(1,2)
        lstm_outs, hidden = self.lstm(x)
        last_out = self.last_timestep(lstm_outs, self.bidirectional)
        output = self.fc(last_out)
        output = self.sigmoid(output)
        return output

    def last_timestep(self, outputs, bidirectional=False):
        """
            Returns the last output of the LSTM taking into account the zero padding
        """
        if bidirectional:
            forward, backward = self.split_directions(outputs)
            last_forward = forward[:, -1, :]
            last_backward = backward[:, 0, :]
            # Concatenate and return - maybe add more functionalities like average
            return torch.cat((last_forward, last_backward), dim=-1)
        else:
            return last_forward

    @staticmethod
    def split_directions(outputs):
        direction_size = int(outputs.size(-1) / 2)
        forward = outputs[:, :, :direction_size]
        backward = outputs[:, :, direction_size:]
        return forward, backward

In [ ]:
D_in, H, D_out, num_layers, dropout, biLSTM = 306, 30, 1, 2, 0.1, True

model = CLSTM(D_in, H, D_out, num_layers, dropout, biLSTM)
model.to(0)

criterion =  nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), weight_decay=0.0001)
model.parameters

In [ ]:
train(train_loader, val_loader, model, criterion, optimizer, max_epochs=50, patience=20)

In [ ]:
predictions, labels = predict(test_loader, model, criterion)

In [ ]:
img_data = ImageDataset()
train_loader, val_loader, test_loader = torch_train_val_split(img_data, 32 ,32, val_size=.2, shuffle=True)

In [ ]:
class Unit(nn.Module):
    def __init__(self,in_channels,out_channels):
        super(Unit,self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channels,kernel_size=5,out_channels=out_channels,stride=1,padding=2)
        self.bn = nn.BatchNorm2d(num_features=out_channels)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()

    def forward(self,x):
        output = self.conv(x.float())
        output = self.bn(output)
        output = self.relu(output)
        output = self.pool(output)
        return output
    
class CNN(nn.Module):
    def __init__(self, num_class):
        super(CNN, self).__init__()
        self.num_class = num_class
        self.unit1 = Unit(in_channels=1, out_channels=16)
        self.unit2 = Unit(in_channels=16, out_channels=8)
        self.unit3 = Unit(in_channels=8, out_channels=4)
        self.unit4 = Unit(in_channels=4, out_channels=2)
        self.net = nn.Sequential(self.unit1, self.unit2, self.unit3, self.unit4)
        self.fc1 = nn.Linear(2*15*19, 1)
        #self.fc2 = nn.Linear(500, self.num_class)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        output = self.net(x)
        output = output.view(output.size(0) , -1)
        output = self.fc1(output)
        #output = self.fc2(output)
        output = self.sigmoid(output)
        return output

In [ ]:
model = CNN(1)
model.to(0)

criterion =  nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), weight_decay=0.005)
model.parameters

In [ ]:
train(train_loader, val_loader, model, criterion, optimizer, max_epochs=100, patience=20)

In [ ]:
predictions, labels = predict(test_loader, model, criterion)